In [1]:
from sentence_transformers import SentencesDataset, InputExample, losses, util
from torch.utils.data import DataLoader

In [2]:
from transformers import AutoModel, AutoTokenizer

bert_model = AutoModel.from_pretrained('./mlm_bert')
bert_tokenizer = AutoTokenizer.from_pretrained('./mlm_bert')

Some weights of BertModel were not initialized from the model checkpoint at ./mlm_bert and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import pandas as pd

example_hash = set()
train_examples = []

sent_pairs_df = pd.read_csv('../data/processed/sent_pairs.csv')

for sent_1, sent_2, final_score in sent_pairs_df[['sent1', 'sent2', 'final_score']].values:
    train_examples.append(
    InputExample(texts=[sent_1, sent_2], label=round(float(final_score), 2))
)

In [4]:
from sentence_transformers import SentenceTransformer, models

# Обертка вашей модели BERT в модуль для SBERT
word_embedding_model = models.Transformer(
    './mlm_bert',
    max_seq_length=256,
    do_lower_case=False,
)

# Создание модели пулинга (pooling). Это сокращает все токенные векторы до одного вектора предложения
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

# Создание объекта SentenceTransformer с использованием модели BERT и модели пулинга
sbert_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of BertModel were not initialized from the model checkpoint at ./mlm_bert and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Создание и загрузка датасета
train_dataset = SentencesDataset(examples=train_examples, model=sbert_model)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Настройка процесса обучения
train_loss = losses.CosineSimilarityLoss(model=sbert_model)

# Обучение модели
# sbert_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1)

In [6]:
sbert_model.save('./sbert_from_mlm_bert')

In [8]:
sentences1 = [
    '1567 гъэм и бжьыхьэм кърым пащтыхьищ, дзэшхуэхэр я гъусэу, адыгэхэм я щIыр къазэуну игъэкIуащ.',
    'КIуащ БетIал 1920 гъэм ноябрым и 22-м Дохъушыкъуей (Старэ Шэрэдж) къуажэм къыщалъхуащ.',
    'Хэку зауэшхуэр къызэрыхъейуэ, 1941 гъэм июным и 24-м Мухьэдин зауэм кIуащ.',
    'Уэ езым болъагъу абы и Iуэху зыIутыр.',
    'хэт уэ уи цIэр?',
]

sentences2 = [
    'бжьыхьэм, 1567 гъэм, кърым пащтыхьищ, дзэшхуэхэр я гъусэу, адыгэхэм я щIыр къазэуну игъэкIуащ.',
    '1920 гъэм етIанэу гвардие хужьыр зэтрикъутащ – генералу 4-рэ офицеру 60-рэ гъэру иубыдащ.',
    '1941 гъэм и октябрым ЩоджэнцIыкIум зауэм дэкIыну зэрыхуеймкIэ лъэIу тхылъ етх.',
    '– къищIащ абы щIалэхэм я Iуэху зыIутыр.',
    'cэ си цIэр Iэдэм',
]

# Compute embedding for both lists
embeddings1 = sbert_model.encode(sentences1, convert_to_tensor=True)
embeddings2 = sbert_model.encode(sentences2, convert_to_tensor=True)

# Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(
        sentences1[i], sentences2[i], cosine_scores[i][i]
    ), end='\n\n')

1567 гъэм и бжьыхьэм кърым пащтыхьищ, дзэшхуэхэр я гъусэу, адыгэхэм я щIыр къазэуну игъэкIуащ. 		 бжьыхьэм, 1567 гъэм, кърым пащтыхьищ, дзэшхуэхэр я гъусэу, адыгэхэм я щIыр къазэуну игъэкIуащ. 		 Score: 0.9358

КIуащ БетIал 1920 гъэм ноябрым и 22-м Дохъушыкъуей (Старэ Шэрэдж) къуажэм къыщалъхуащ. 		 1920 гъэм етIанэу гвардие хужьыр зэтрикъутащ – генералу 4-рэ офицеру 60-рэ гъэру иубыдащ. 		 Score: 0.7052

Хэку зауэшхуэр къызэрыхъейуэ, 1941 гъэм июным и 24-м Мухьэдин зауэм кIуащ. 		 1941 гъэм и октябрым ЩоджэнцIыкIум зауэм дэкIыну зэрыхуеймкIэ лъэIу тхылъ етх. 		 Score: 0.6028

Уэ езым болъагъу абы и Iуэху зыIутыр. 		 – къищIащ абы щIалэхэм я Iуэху зыIутыр. 		 Score: 0.8375

хэт уэ уи цIэр? 		 cэ си цIэр Iэдэм 		 Score: 0.5879

